In [1]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
from collections import OrderedDict, defaultdict
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import numpy as np
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
imacscrapedir = '/Users/rayd/workspace/cinema/scrapes/'
airscrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-07-11.csv'
scrape_path = airscrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   
      
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
#df = df[df.city != "Belleview, FL"]
#df = df[df.city != "High Springs, FL"]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
#df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
# fix up titles, times and days, keep just needed columns, sort rows by movie+day
df.title = df.title.apply(lambda x: '"' + str(x) + '"')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace('p', ' p.m.')
df.times = df.times.str.replace('a', ' a.m.,')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df = df[['theater', 'screen', 'movieday', 'times']]
df.sort_values(["screen", "movieday"], axis =0,
              ascending=True, inplace=True)
df = df.drop_duplicates()

# Ocala market
oc6_df = df.loc[df['theater'].str.contains('Ocala Center 6')]
ocd_df = df.loc[df['theater'].str.contains('Drive-In')]
bv_df = df.loc[df['theater'].str.contains('Belleview')]
mt_df = df.loc[df['theater'].str.contains('Marion')]
omp_df = df.loc[df['theater'].str.contains('Old Mill')]
regaloca_df = df.loc[df['theater'].str.contains('Regal Hollywood')]
rialto_df = df.loc[df['theater'].str.contains('Rialto')]
amc_df = df.loc[df['theater'].str.contains('AMC Lake')]
barn_df = df.loc[df['theater'].str.contains('Barnstorm')]

# Gainesville market
celeb_df = df.loc[df['theater'].str.contains('Celebration')]
hipp_df = df.loc[df['theater'].str.contains('Hippodrome')]
butler_df = df.loc[df['theater'].str.contains('Butler')]
regalgnv_df = df.loc[df['theater'].str.contains('Regal Royal Park')]
uac_df = df.loc[df['theater'].str.contains('Regal UA Cinema 90')]
starke_df = df.loc[df['theater'].str.contains('Twin')]


In [2]:
# Convered to ordered dict, and then to list of ordered dicts
doc6 = oc6_df.to_dict(into=OrderedDict)
items = list(doc6.items())

In [3]:
# Convert ordered dicts to lists of movies, days and times
movievals = list(items[1][1].values())
dayvals = list(items[2][1].values())
timevals = list(items[3][1].values())

In [4]:
movieday = ['; '.join(map(str, i)) for i in zip(movievals, dayvals)]
moviedaytime = ['; '.join(map(str, i)) for i in zip(movieday, timevals)]

In [5]:
print(len(movievals))
print(len(dayvals))
print(len(timevals))

18
18
18


In [10]:
n = 3
del movievals[:n]
del dayvals[:n]
del timevals[:n]
print(len(movievals))
print(len(dayvals))
print(len(timevals))

3
3
3


In [ ]:
while len(movievals) > 1:
    if len(movievals) >= 3:
        if movievals[0] == movievals[1] and movievals[1] == movievals[2]:
            print("Runs Friday to Sunday")
            print("There are " + str(len(movievals)) + " items left.\n")
            
            # Run the logic for consolidating 3 days and times.
            # Add the text to that to a file.
            # Delete all three items from all three lists.
                       
            n = 3
            del movievals[:n]
            del dayvals[:n]
            del timevals[:n]
        
        elif movievals[0] == movievals[1] and movievals[1] != movievals[2]:
            print("Could be Friday and Saturday, or Saturday and Sunday")
            if dayvals[0] == "Friday" and dayvals[1] == "Saturday":
                print("It runs Friday and Saturday")
                print("There are " + str(len(movievals)) + " items left.\n")

                # Run the logic for consolidating Friday and Saturday.
                # Add the text to a file.
                # Delete two items from all three lists.
                                
                n = 2
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                                                              
            elif dayvals[0] == "Saturday" and dayvals[1] == "Sunday":
                print("Saturday and Sunday Sunday run")
                print("There are " + str(len(movievals)) + " items left.\n")

                # Run the logic for consolidating Saturday and Sunday.
                # Add text to a file.
                # Delete two rows from all three lists.
                                    
                n = 2
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                
            elif dayvals[0] == "Friday" and dayvals[1] == "Sunday":
                print("Friday and Sunday -- very odd -- better check!")
                print("There are " + str(len(movievals)) + " items left.\n")
                
                # Run the logic for consolidating Friday and Sunday.
                # Add some sort of alert or break.
                # Add text to a file.
                # Delete two items from all three lists.
                
                n = 2
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                
            else:
                print("Eek!")
                break
                
        else:
            print("One-day run: Friday, Saturday or Sunday")
            print("There are " + str(len(movievals)) + " items left.\n")
            
            # Run logic for which day.
            # Add text to file.
            # Delete one item from all three lists.
            
            n = 1
            del movievals[:n]
            del dayvals[:n]
            del timevals[:n]
    
    # Len must = 2
    if movievals[0] == movievals[1]:
        print("Could be Friday and Saturday, or Saturday and Sunday")
        print("There are " + str(len(movievals)) + " items left.\n")

        if dayvals[0] == "Friday" and dayvals[1] == "Saturday":
            print("It runs Friday and Saturday")
                        
            # Run the logic for consolidating Friday and Saturday.
            # Add the text to a file.
            # Delete two items from all three lists.
            
            
            n = 2
            del movievals[:n]
            del dayvals[:n]
            del timevals[:n] 
            
    else:
        print("It's a one-day run, but there is still one day at this theater.")
        print("There are " + str(len(movievals)) + " items left.\n")

        # Run logic to find out which day and time.
        # Add text to file.
        # Delete one item from each list"""

        n = 1
        del movievals[:n]
        del dayvals[:n]
        del timevals[:n]
        
print("Here's what's left after the loop ran:")
print(movievals)
print(dayvals)
print(timevals)

In [ ]:
movievals

In [ ]:
"""
Once it's working for one threater, put all the above
into a function that can run for each theater, and 
write for files for each day for Ocala and Gainesville.
"""